# Using Pyrea with Nutrimouse Data Utilising Hierarchical and Spectral Clustering

In this notebok we demonstrate Pyrea's usage by performing hierarchical and spectral clustering on the Nutrimouse[<sup>1</sup>](#fn1) dataset.

We will do this using the Parea_1 structure, a structure that is included as a helper function in the Pyrea software package.

## Imports
This notebook requires Pyrea, mvlearn, Numpy, and sklearn (for computing metrics). Let's import the relevant items here:

In [14]:
import pyrea
import numpy as np
from mvlearn.datasets import load_nutrimouse
from mvlearn.cluster import MultiviewCoRegSpectralClustering, MultiviewSphericalKMeans, MultiviewKMeans

from sklearn.metrics import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_mutual_info_score as ami_score

## Load Data

Load the Nutrimouse data using mvlearn:[<sup>2</sup>](#fn2)

In [2]:
nutrimouse_dataset = load_nutrimouse()
data = [nutrimouse_dataset['gene'], nutrimouse_dataset['lipid']]

y_all = np.vstack((nutrimouse_dataset['genotype'], nutrimouse_dataset['diet'])).T
y = y_all[:,0]

Note: the variable `y` contains the ground truths that we will use for evaluation later in the notebook. The ground truths are not used during training.

Preview the shape of the data:

In [3]:
print(f'Number of views: {len(data)}')
print(f'Shape of view 1: {np.shape(data[0])[0]} x {np.shape(data[0])[1]}')
print(f'Shape of view 2: {np.shape(data[1])[0]} x {np.shape(data[1])[1]}')

Number of views: 2
Shape of view 1: 40 x 120
Shape of view 2: 40 x 21


As can be seen there are 2 views. View 1 has 120 features for each of the 40 mice, while view 2 has 21 features for each of the 40 mice. As this is a multi-view dataset, the 40 samples refer to the same 40 mice in both datasets.

We will use Parea_1 to perform hierarchical clustering and spectral clustering on this dataset, and use Pyrea's built-in genetic algorithm functionality to find the best hyperparameters to use for this data.

# Parea_1

## Hierarchical Clustering

In this section, we will execute the Parea_1 structure using both hierarchical clustering and spectral clustering. We will optimise this structure's parameters using a genetic algorithm.

First we use hierarchical clustering, and use Pyrea's genetic algorithm helper function for this: `parea_1_genetic()`.

We execute the genetic algorithm as follows, which will learn the best parameters to use for the clustering:

In [69]:
params_hierarchical = pyrea.parea_1_genetic(data, k_min=2, k_max=5, k_final=2, n_generations=10, n_population=10)

Silhouette score: 0.55
Silhouette score: 0.6802083333333334
Silhouette score: 0.623624835309618
Silhouette score: 0.6488505747126437
Silhouette score: 0.7221413255360624
Silhouette score: 0.5850405546834118
Silhouette score: 0.5891369047619047
Silhouette score: 0.6802083333333334
Silhouette score: 0.4802196275946276
Silhouette score: 0.7201178451178453
gen	nevals	avg     	std      	min    	max     
0  	10    	0.627955	0.0738461	0.48022	0.722141
Silhouette score: 0.6802083333333334
Silhouette score: 0.7201178451178453
Silhouette score: 0.7221413255360624
Silhouette score: 0.623624835309618
Silhouette score: 0.7046678406875776
Silhouette score: 0.7020833333333334
Silhouette score: 0.7201178451178453
Silhouette score: 0.6802083333333334
1  	8     	0.685833	0.0442218	0.585041	0.722141
Silhouette score: 0.6064814814814815
Silhouette score: 0.84375
Silhouette score: 0.6802083333333334
Silhouette score: 0.7046678406875776
2  	4     	0.705998	0.0561702	0.606481	0.84375 
Silhouette score: 0.720

Once this is complete `params_hierarchical` contains the optimal parameters for this data, which we can then use to call the `parea_1` function with the optimal parameters that we have learned:

In [70]:
labels_hierarchical = pyrea.parea_1(data, *params_hierarchical)

print(labels_hierarchical)

[1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


Compare this to the true labels:

In [57]:
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


We can also print the parameters to see which were selected by the genetic algorithm:

In [58]:
params_hierarchical

['hierarchical',
 'ward2',
 2,
 'hierarchical',
 'ward',
 2,
 'hierarchical',
 'centroid',
 2,
 'hierarchical',
 'complete',
 2,
 'disagreement']

In the [Evaluation](#Evaluation) section below, we will use the Normalized Mutual Information (NMI) score to judge how well it performed, and compare it to some specialised multi-view algorithms provided by mvlearn.

## Spectral Clustering

In this section we will again use the Parea_1 structure, but using spectral clustering.

This is performed in almost the same way:

In [40]:
params_spectral = pyrea.parea_1_genetic_spectral(data, k_min=2, k_max=8, k_final=2, n_neighbors_min=10, n_neighbors_max=15, n_population=10, n_generations=3)

Silhouette score: 0.45367647058823535
Silhouette score: 0.30891873278236914
Silhouette score: 0.41153447933019366
Silhouette score: 0.25472972972972974
Silhouette score: 0.7732254678716288
Silhouette score: 0.28855043345019765
Silhouette score: 0.38894880321678194
Silhouette score: 0.8402388542273137
Silhouette score: 0.7521623496762256
Silhouette score: 0.5664062592891516
gen	nevals	avg     	std     	min    	max     
0  	10    	0.503839	0.205414	0.25473	0.840239
Silhouette score: 0.7715909090909091
Silhouette score: 0.6631399853068858
Silhouette score: 0.5753395004625347
Silhouette score: 0.8887608695652176
Silhouette score: 0.6887356355067148
Silhouette score: 0.5149516463562169
Silhouette score: 0.7883522727272727
Silhouette score: 0.8568979933110368
Silhouette score: 0.6189460516356908
1  	9     	0.682039	0.137283	0.453676	0.888761
Silhouette score: 0.7142241379310346
Silhouette score: 0.45435703334344824
Silhouette score: 0.4781618665401342
Silhouette score: 0.6974738070835633
Sil

Again this returns our optimal parameters, which we then use to perform the clustering using the `parea_1_spectral` function:

In [45]:
labels_spectral = pyrea.parea_1_spectral(data, *params_spectral, k_final=2)

print(labels_spectral)

[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


Compare these to the true labels:

In [46]:
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


We can also print the parameters to see which were selected by the genetic algorithm:

In [47]:
params_spectral

['spectral',
 15,
 4,
 'spectral',
 12,
 4,
 'spectral',
 15,
 2,
 'spectral',
 12,
 2,
 'agreement']

Now we perform a proper evaluation of the predicted in the [Evaluation](#Evaluation) section below.

# Evaluation

## Parea_1

We compare our results from Parea_1 to several specially designed algorithms for multi-view, using mvlearn. We use the Normalized Mutual Information (NMI) score for this evaluation.

In this section we will evaluate Parea_1 using hierarchical clustering and then using spectral clustering.

First we compute the NMI score for hierarchical clustering (using ther ground truths stored in `y` from above):

In [72]:
s_nmi_hierarchical = nmi_score(labels_hierarchical, y)
s_nmi_hierarchical

0.5615896365639194

And now for spectral clustering:

In [48]:
s_nmi_spectral = nmi_score(labels_spectral, y)
s_nmi_spectral

0.6843628400956292

## mvlearn

Here we use mvlearn's specialised multi-view clustering algorithms on the same data and also compute the NMI scores. 

First, Multiview Coregularized Spectral Clustering:

In [49]:
mv_coreg_spectral = MultiviewCoRegSpectralClustering(n_clusters=2,
                                               random_state=42,
                                               n_init=100)

labels_mv_coreg_spectral = mv_coreg_spectral.fit_predict(data)

s_nmi_mv_coreg_spectral = nmi_score(labels_mv_coreg_spectral, y)
s_nmi_mv_coreg_spectral

0.4591921489796446

Now with Multiview Spherical Means:

In [14]:
mv_spherical_kmeans = MultiviewSphericalKMeans(n_clusters=2, random_state=42)

labels_spherical_kmeans = mv_spherical_kmeans.fit_predict(data)

s_nmi_spherical_kmeans = nmi_score(labels_spherical_kmeans, y)
s_nmi_spherical_kmeans

0.6843628400956293

And finally we use mvlearn's Multiview KMeans:

In [15]:
mv_kmeans = MultiviewKMeans(n_clusters=2, random_state=42)

labels_kmeans = mv_kmeans.fit_predict(data)

s_nmi_kmeans = nmi_score(labels_kmeans, y)
s_nmi_kmeans

0.5615896365639194

All results are summarised at the end of this notebook for comparison.

# Parea 2

In this section, we will run the Parea 2 structure using hierarchical clustering and optimising its hyperparameters with a genetic algorithm. We will then evaluate its performance using the Normalized Mutual Information score as above.

In [109]:
params_parea2 = pyrea.parea_2_mv_genetic(data, k_min=2, k_max=6, n_population=100, n_generations=5)

['c_0_type', 'c_1_type', 'c_0_method', 'c_1_method', 'c_0_k', 'c_1_k', 'c_0_pre_type', 'c_1_pre_type', 'c_0_pre_method', 'c_1_pre_method', 'c_0_pre_k', 'c_1_pre_k', 'fusion_method']
Silhouette score: 0.25206614153538387
Silhouette score: 0.3279878618113912
Silhouette score: 0.3299924287374579
Silhouette score: 0.2042559277639923
Silhouette score: 0.34075598421186654
Silhouette score: 0.2391629157289322
Silhouette score: 0.3565592903828198
Silhouette score: 0.2042559277639923
Silhouette score: 0.2391629157289322
Silhouette score: 0.529158739073293
Silhouette score: 0.440625
Silhouette score: 0.4859592299809691
Silhouette score: 0.3279878618113912
Silhouette score: 0.25206614153538387
Silhouette score: 0.4024972943722943
Silhouette score: 0.213933347118831
Silhouette score: 0.40583323479485606
Silhouette score: 0.529158739073293
Silhouette score: 0.32025540222507487
Silhouette score: 0.34075598421186654
Silhouette score: 0.4024972943722943
Silhouette score: 0.4859592299809691
Silhouette 

The genetic algorithm training is complete.

We can take a look at the parameters that were chosen by the genetic algorithm:

In [110]:
params_parea2

{'clusterers': ['hierarchical', 'hierarchical'],
 'methods': ['ward2', 'single'],
 'k_s': [2, 3],
 'precomputed_clusterers': ['hierarchical', 'hierarchical'],
 'precomputed_methods': ['complete', 'weighted'],
 'precomputed_k_s': [2, 2],
 'fusion_method': 'disagreement'}

To get the score, we can run Parea_2 using the optimal parameters, which returns the predicted labels:

In [111]:
labels_parea_2 = pyrea.parea_2_mv(data, **params_parea2)
print(labels_parea_2)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
 0 0 0]


Compare this with the ground truth:

In [112]:
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


Now we can use the predicted labels to create the NMI score:

In [113]:
s_nmi_parea_2 = nmi_score(labels_parea_2, y)
print(f"NMI score: {s_nmi_parea_2}")

NMI score: 0.6843628400956293


# Conclusions

The best scores obtained are summarised below (note these may differ from the scores above, as this table contains the **best** scores achieved):

| Method                                                | NMI Score              |
|-------------------------------------------------------|------------------------|
| Parea_1 (Hierarchical)                                | 0.5615896365639194     |
| **Parea_1 (Spectral)**                                | **0.6843628400956292** |
| **Parea_2 (Hierarchical)**                            | **0.6843628400956293** |
| mvlearn (Multiview Coregularized Spectral Clustering) | 0.4591921489796446     |
| **mvlearn (Multiview Spherical Means)**               | **0.6843628400956293** |
| mvlearn (Multiview KMeans)                            | 0.5615896365639194     |

## Configurations

Here we desribe the configurations that led the scores above, for Parea_1 and Parea_2.

### Parea_2 (Hierarchical)

Parea_2 with hierarcical clustering scores 0.6843628400956293 with following training configuration: 

```python
params_parea2 = pyrea.parea_2_mv_genetic(data, k_min=2, k_max=6, n_population=100, n_generations=5)
```

Resulting in the following parameters:

```js
{'clusterers': ['hierarchical', 'hierarchical'],
 'methods': ['ward2', 'single'],
 'k_s': [2, 3],
 'precomputed_clusterers': ['hierarchical', 'hierarchical'],
 'precomputed_methods': ['complete', 'weighted'],
 'precomputed_k_s': [2, 2],
 'fusion_method': 'disagreement'}
```

### Parea_1 (Spectral) 

Parea_1 with spectral clustering scores 0.6843628400956292 with the following training configuration:

```python
params_spectral = pyrea.parea_1_genetic_spectral(data, k_min=2, k_max=8, k_final=2, n_neighbors_min=10, n_neighbors_max=15, n_population=10, n_generations=3)
```

Resulting in the following parameters: 

```js
['spectral', 15, 4, 'spectral', 12, 4, 'spectral', 15, 2, 'spectral', 12, 2, 'agreement']
```

### Parea_1 (Hierarchical)

Parea_1 with hierarchical clustering scores 0.5615896365639194 with the following training configuration:

```python
params_hierarchical = pyrea.parea_1_genetic(data, k_min=2, k_max=5, k_final=2, n_generations=10, n_population=10)
```

Resulting in the following parameters:

```js
['hierarchical',
 'ward2',
 2,
 'hierarchical',
 'ward',
 2,
 'hierarchical',
 'centroid',
 2,
 'hierarchical',
 'complete',
 2,
 'disagreement']
```

---

_Note that the values in the table above are the best scores achieved, and may be different when the notebook is re-run. Due to the stochastic nature of the genetic algorithm, new runs will produce different clusterings and different NMI scores._

---

# References

<span id="fn1"><sup>1</sup> Nutrimouse data: https://aasldpubs.onlinelibrary.wiley.com/doi/10.1002/hep.21510</span>

<span id="fn2"><sup>2</sup> mvlearn Nutrimouse example: https://mvlearn.github.io/auto_examples/datasets/plot_nutrimouse.html</span>